### "Simplification" attemps with OSMnx

For test (continents) cities [FUA ID]:
* (Africa) Douala [809]
* (Oceania) Auckland [869]
* (Asia) Aleppo [1133] -- Wuhan [8989]
* (Europe) Liège [1656]
* (South America) Bucaramanga [4617]
* (North America) Salt Lake City [4881]

In [1]:
%load_ext watermark
%watermark

Last updated: 2024-04-16T21:36:44.149228-04:00

Python implementation: CPython
Python version       : 3.11.8
IPython version      : 8.22.2

Compiler    : Clang 16.0.6 
OS          : Darwin
Release     : 23.4.0
Machine     : arm64
Processor   : arm
CPU cores   : 8
Architecture: 64bit



In [2]:
# import libraries
import pathlib

import contextily as cx
import cv2
import geopandas as gpd
import matplotlib.pyplot as plt
import momepy
import osmnx as ox
import utils
from shapely.geometry import Point

%watermark -w
%watermark -iv

Watermark: 2.4.3

cv2       : 4.9.0
json      : 2.0.9
osmnx     : 1.9.2
contextily: 1.6.0
geopandas : 0.14.3
momepy    : 0.7.1.dev23+ga5e30d8
matplotlib: 3.8.4



**Read in meta data**

In [3]:
# read in sample metadata
sample = utils.read_sample_data()
sample.head(2)

,eFUA_ID,UC_num,UC_IDs,eFUA_name,Commuting,Cntry_ISO,Cntry_name,FUA_area,UC_area,FUA_p_2015,UC_p_2015,Com_p_2015,geometry,continent,iso_a3
305,9129.0,1.0,8078,Gonda,1.0,IND,India,66.0,29.0,1.074100e+06,1.066419e+06,7680.678101,"POLYGON ((81.98398 27.19657, 81.99471 27.19657...",Asia,IND
91,7578.0,6.0,10577;10581;10583;10596;10605;10607,Chongqing,1.0,CHN,China,2267.0,618.0,6.036834e+06,5.157726e+06,879107.861057,"POLYGON ((106.23972 29.52328, 106.19622 29.523...",Asia,CHN


In [4]:
utils.city_fua

{'Aleppo': 1133,
 'Auckland': 869,
 'Bucaramanga': 4617,
 'Douala': 809,
 'Liège': 1656,
 'Salt Lake City': 4881,
 'Wuhan': 8989}

**Read in data for example city**

In [5]:
# read in data for example city: Liege
city = "Liège"
gdf = utils.read_parquet_roads(utils.city_fua[city])
G = momepy.gdf_to_nx(
    gdf_network=gdf, approach="primal", directed=True, integer_labels=True
)
print(utils.graph_size(f"Momepy-NetworkX Primal Graph: {city}", G))

Momepy-NetworkX Primal Graph: Liège
	* MultiDiGraph with 98777 nodes and 105174 edges


**Simplify graph (in OSMNnx terms, i.e. remove interstitial nodes)**

In [6]:
G_simp = ox.simplify_graph(G)
print(utils.graph_size(f"Momepy-NetworkX Primal Graph (simplified): {city}", G_simp))

Momepy-NetworkX Primal Graph (simplified): Liège
	* MultiDiGraph with 38034 nodes and 44421 edges


In [7]:
# check crs, needs to be a projected one
G_simp.graph["crs"]

<Projected CRS: {"$schema": "https://proj.org/schemas/v0.7/projjso ...>
Name: unknown
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: UTM zone 31N
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

**Consolidate nodes (test different thresholds)**

In [8]:
cons_dict = {}
for tol in range(1, 21):
    # consolidate graph
    G_cons = ox.consolidate_intersections(
        G=G_simp,
        tolerance=tol,
        rebuild_graph=True,
        dead_ends=True,
        reconnect_edges=True,
    )
    # derive consolidated edges
    edges_cons = ox.graph_to_gdfs(G=G_cons, nodes=False, edges=True)
    # save in cons_dict
    cons_dict[tol] = {}
    cons_dict[tol]["graph"] = G_cons
    cons_dict[tol]["edges"] = edges_cons
    print(utils.graph_size(f"Consolidating with tolerance {tol}m", G_cons))

Consolidating with tolerance 1m
	* MultiDiGraph with 37899 nodes and 44285 edges
Consolidating with tolerance 2m
	* MultiDiGraph with 37286 nodes and 43670 edges
Consolidating with tolerance 3m
	* MultiDiGraph with 36165 nodes and 42538 edges
Consolidating with tolerance 4m
	* MultiDiGraph with 34769 nodes and 41125 edges
Consolidating with tolerance 5m
	* MultiDiGraph with 33210 nodes and 39527 edges
Consolidating with tolerance 6m
	* MultiDiGraph with 31579 nodes and 37824 edges
Consolidating with tolerance 7m
	* MultiDiGraph with 30058 nodes and 36205 edges
Consolidating with tolerance 8m
	* MultiDiGraph with 28322 nodes and 34353 edges
Consolidating with tolerance 9m
	* MultiDiGraph with 26561 nodes and 32437 edges
Consolidating with tolerance 10m
	* MultiDiGraph with 25158 nodes and 30847 edges
Consolidating with tolerance 11m
	* MultiDiGraph with 23902 nodes and 29445 edges
Consolidating with tolerance 12m
	* MultiDiGraph with 22771 nodes and 28187 edges
Consolidating with tolera

In [9]:
for tol in cons_dict:
    cons_dict[tol]["nodes"] = ox.graph_to_gdfs(
        G=cons_dict[tol]["graph"], nodes=True, edges=False
    )

**Find examples** (see nb `usecases.ipynb`)

**Check what happens for examples / use cases for different consolidation thresholds**

Make plots and videos for each use case, for gradually increasing simplification threshold

In [10]:
points, fpath_base = utils.load_usecases(city)
fpath_base

PosixPath('../usecases/1656')

In [11]:
package = "osmnx"
fpath_pack = fpath_base / package
fpath_pack

PosixPath('../usecases/1656/osmnx')

In [12]:
for _class, v in points.items():
    mypoint = v["coords"]
    myclass = _class

    # make subfolder for plot saving
    fpath_class = fpath_pack / myclass
    fpath_class.mkdir(parents=True, exist_ok=True)

    # get center frame (for clipping)
    center = gpd.GeoDataFrame(geometry=[Point(mypoint)], crs="epsg:4326")
    center = center.to_crs(gdf.crs)
    center = center.buffer(250, cap_style=3)

    # -------------------------------------------------------------------
    # for each tolerance threshold
    # MAKE PLOTS ------------------------------------------
    for tol in cons_dict:
        # make a plot
        fig, ax = plt.subplots(1, 1, figsize=(8, 8))

        edges = cons_dict[tol]["edges"]
        nodes = cons_dict[tol]["nodes"]

        # clip geometries to box
        edges_clipped = edges.copy()
        nodes_clipped = nodes.copy()
        edges_clipped = edges_clipped.clip(center)
        nodes_clipped = nodes_clipped.clip(center)

        # plot
        edges_clipped.plot(ax=ax, zorder=1, color="black", linewidth=2)
        nodes_clipped.plot(ax=ax, zorder=2, color="red", markersize=15, alpha=0.9)
        cx.add_basemap(ax=ax, source=cx.providers.CartoDB.Voyager, crs=gdf.crs)
        ax.set_axis_off()
        ax.set_title(f"Tolerance {tol}m")
        plt.tight_layout()

        # save to subfolder
        fill_tol = f"{tol:03d}"
        fig.savefig(fpath_class / f"{fill_tol}.png", dpi=300)
        plt.close()

    # MAKE VIDEOS -----------------------------------------
    fps = 1
    images = sorted(fpath_class.glob("*.png"))
    video_name = pathlib.Path(f"{fpath_class}.mp4")
    frame = cv2.imread(str(images[0]))
    height, width, layers = frame.shape
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    video = cv2.VideoWriter(str(video_name), fourcc, fps, (width, height))
    for image in images:
        video.write(cv2.resize(cv2.imread(str(image)), (width, height)))
    cv2.destroyAllWindows()
    video.release()